**Install the required modules and libraries**

upload a txt file and paste these modules in it

accelerate == 0.31.0

bitsandbytes == 0.43.1

transformers == 4.42.3

In [ ]:
!pip install -r requirements.txt  # accelerate == 0.31.0
                                  #bitsandbytes == 0.43.1
                                  #transformers == 4.42.3

In [ ]:
import pandas as pd
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
from sklearn.metrics import f1_score
!pip install rouge-score
from rouge_score import rouge_scorer
data = pd.read_json("test.json")

In [ ]:
HF_TOKEN = #paste your own Huggingface token in here after getting the gated model access

*https://huggingface.co/models*

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
#setting quantization parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

In [ ]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.01,
    top_p=0.95,
    repetition_penalty=1.15,
    top_k = 1,
    #echo = False
)

In [ ]:
data1 = data.iloc[0]

prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. Always give factually coherent answers. Keep your answers to be brief within 3 sentences.

USER: {data1['question']}

CONTEXT: {data1['context']}

ASSISTANT:
'''
response = generator(prompt_template)

In [ ]:
generated_text = response[0]["generated_text"]
if "ASSISTANT:" in generated_text:
    assistant_response = generated_text.split("ASSISTANT:")[1].strip().split("\n")[0].strip()
else:
    assistant_response = generated_text.strip()

print(assistant_response)

In [ ]:
data2 = data.iloc[5]
data8 = data.iloc[8]
prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. Interpret the context given carefully.  Always give factually coherent answers. Always give an answer.
Try to be mathematically correct.
USER: {data2['question']}

CONTEXT: {data2['context']}

ASSISTANT: {data2['answers']}

User: {data8['question']}

Context: {data8['context']}

Assistant:
'''
response = generator(prompt_template)

In [ ]:
generated_text = response[0]["generated_text"]
if "ASSISTANT:" in generated_text:
    assistant_response = generated_text.split("Assistant:")[1].split("USER:")[0].strip().replace('\n', ' ').strip().split("##")[0].strip().replace('\n', ' ')
else:
    assistant_response = generated_text.strip().replace('\n', ' ')

assistant_response

In [ ]:
# Define the few-shot examples
#picking random examples from the dataset
prom = data.iloc[100]
promp = data.iloc[101]
promt = data.iloc[102]
few_shot_examples = [
    {
        "question": prom['question'],
        "context": prom['context'],
        "answer": prom['answers']
    },
    {
        "question": promp['question'],
        "context": promp['context'],
        "answer": promp['answers']
    },
    {
        "question": promt['question'],
        "context": promt['context'],
        "answer": promt['answers']
    },
]

In [ ]:
# Define the new question and context
data9 = data.iloc[9]
new_question = data9['question']
context = data9['context']

In [ ]:
# Construct the input prompt
def construct_prompt(examples, question, context):
    prompt = "Answer the following questions based on the context provided.\n\n"
    for example in examples:
        prompt += f"Question: {example['question']}\nAnswer: {example['answer']}\n\n"
    prompt += f"Context: {context}\nQuestion: {question}\nAnswer:"
    return prompt
prompt = construct_prompt(few_shot_examples, new_question, context)

In [ ]:
# Generate the answer
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)

# Extract and print the generated answer
generated_text = result[0]['generated_text']
clean_text = generated_text.split('Answer:')[-1].strip()

In [ ]:
clean_text

In [ ]:
data_sample = data.head(3)

model_responses = []
for index, row in data_sample.iterrows():
    prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. Interpret the context given carefully.  Always give factually coherent answers. Always give an answer.

USER: {row['question']}

CONTEXT: {row['context']}

ASSISTANT:
'''
    response = generator(prompt_template)
    generated_text = response[0]["generated_text"]
    if "ASSISTANT:" in generated_text:
        assistant_response = generated_text.split("ASSISTANT:")[1].split("USER:")[0].strip().replace('\n', ' ').strip().split("##")[0].strip().replace('\n', ' ').strip().split("SYSTEM:")[0].strip().replace('\n', ' ')
    else:
        assistant_response = generated_text.strip().replace('\n', ' ')
    model_responses.append(assistant_response)

data_sample['model_response'] = model_responses

In [ ]:
# Apply ROUGE scores
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []
for true_answer_list, model_answer in zip(data_sample['answers'], data_sample['model_response']):
    # Assuming 'true_answer_list' is a list of strings, join them into a single string
    true_answer = ' '.join(true_answer_list)
    scores = rouge_scorer_instance.score(true_answer, model_answer)
    rouge_scores.append(scores)

data_sample['rouge_score'] = rouge_scores


In [ ]:
# Calculate average scores
avg_rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)

print(f'Average ROUGE-1 F1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 F1 Score: {avg_rouge2}')
print(f'Average ROUGE-L F1 Score: {avg_rougeL}')

In [ ]:
# Apply ROUGE scores
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []
for true_answer_list, model_answer in zip(data_sample['context'], data_sample['model_response']):
    # Assuming 'true_answer_list' is a list of strings, join them into a single string
    true_answer = ' '.join(true_answer_list)
    scores = rouge_scorer_instance.score(true_answer, model_answer)
    rouge_scores.append(scores)

data_sample['rouge_score'] = rouge_scores

In [ ]:
# Calculate average scores
avg_rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)

print(f'Average ROUGE-1 F1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 F1 Score: {avg_rouge2}')
print(f'Average ROUGE-L F1 Score: {avg_rougeL}')

In [ ]:
for index, row in data_sample.iterrows():
    print(f"Question: {index+1}:")
    print(f"Question: {row['question']}")
    print(f"Context: {row['context']}")
    print(f"Model Response: {row['model_response']}")
    print(f"True Answers: {row['answers']}")


